In [39]:
import numpy as np
import pandas as pd
from keras import Sequential
from keras.src.callbacks import ModelCheckpoint
from keras.src.layers import Dense
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.express as px

In [42]:
data = load_breast_cancer()

labels = data.target
data = data.data

In [43]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2)

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [47]:
model = Sequential()
model.add(Dense(units=1, input_shape=(30,), activation="sigmoid"))

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 1)                 31        
                                                                 
Total params: 31 (124.00 Byte)
Trainable params: 31 (124.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
mc = ModelCheckpoint("best_model_breast.keras", save_best_only=True, monitor="val_accuracy", mode="max", verbose=1)

model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

history = model.fit(train_data, train_labels, epochs=100, batch_size=32, callbacks=[mc], validation_split=0.2)

model.save("model_after_training.keras")

Epoch 1/100
 1/12 [=>............................] - ETA: 12s - loss: 0.0477 - accuracy: 1.0000
Epoch 1: val_accuracy improved from -inf to 0.98901, saving model to best_model_breast.keras
12/12 [==============================] - 2s 39ms/step - loss: 0.0711 - accuracy: 0.9808 - val_loss: 0.0595 - val_accuracy: 0.9890
Epoch 2/100
 1/12 [=>............................] - ETA: 0s - loss: 0.0705 - accuracy: 1.0000
Epoch 2: val_accuracy did not improve from 0.98901
12/12 [==============================] - 0s 9ms/step - loss: 0.0709 - accuracy: 0.9808 - val_loss: 0.0596 - val_accuracy: 0.9890
Epoch 3/100
 1/12 [=>............................] - ETA: 0s - loss: 0.0957 - accuracy: 0.9688
Epoch 3: val_accuracy did not improve from 0.98901
12/12 [==============================] - 0s 9ms/step - loss: 0.0707 - accuracy: 0.9808 - val_loss: 0.0592 - val_accuracy: 0.9890
Epoch 4/100
 1/12 [=>............................] - ETA: 0s - loss: 0.1004 - accuracy: 0.9375
Epoch 4: val_accuracy did not improv

In [59]:
def draw_history(history):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["accuracy"], name="accuracy"))
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["val_accuracy"], name="val_accuracy"))
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["loss"], name="loss"))
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["val_loss"], name="val_loss"))
    fig.show()


draw_history(history)

In [62]:
model.load_weights("best_model_breast.keras")

preds = model.predict(test_data)
preds = np.round(preds)

df = pd.DataFrame({
    "true": test_labels,
    "preds": preds.ravel()
})

df

4/4 [==============================] - 0s 3ms/step


,true,preds
0,0,0.0
1,1,1.0
2,1,1.0
3,1,1.0
4,1,1.0
...,...,...
109,1,1.0
110,0,0.0
111,1,1.0
112,1,1.0


In [63]:
print(f"Accuracy: {accuracy_score(df['true'], df['preds']) * 100}%")

cm = confusion_matrix(df['true'], df['preds'])
px.imshow(cm, text_auto=True)

Accuracy: 98.24561403508771%
